In [8]:
import plotly.express as px
import plotly
plotly.io.templates.default = 'plotly_dark'
import pandas as pd
import sqlalchemy as sal

In [9]:
host="capstone.clihskgj8i7s.us-west-2.rds.amazonaws.com"
username="group3"
db="db1"
#pw=getpass.getpass("Enter database password")
pw=open("/home/daniel/Desktop/.awsdb","r").read().rstrip()

In [10]:
engine = sal.create_engine('postgresql://%s:%s@%s/%s' % (username, pw, host, db))

In [11]:
#download a snapshot of data
query="""
select l.sid, l.longitude, l.latitude, l.stype, l.fwy ,l.direc, l.abs_pm, t.total_flow ,
t.avg_speed
from traffic t, pemslocs l
where t.timestamp between '2021-01-22 15:00:00' and '2021-01-22 15:03:00'
and l.sid = t.station and l.stype = 'ML'
"""
snap=pd.read_sql(query, engine, index_col="sid")
snap.head()

,longitude,latitude,stype,fwy,direc,abs_pm,total_flow,avg_speed
sid,,,,,,,,
1108285,-117.134145,32.713318,ML,94,W,0.819,232,64.8
1108287,-117.127908,32.714211,ML,94,W,1.205,153,58.6
1108289,-117.119831,32.716724,ML,94,W,1.721,252,62.9
1108291,-117.117861,32.717134,ML,94,W,1.844,311,65.1
1108293,-117.113171,32.717909,ML,94,W,2.134,277,62.8


In [12]:
pd.read_sql("select * from pemslocs limit 1", engine)
#print(len(snap))

,sid,fwy,direc,district,county,city,state_pm,abs_pm,latitude,longitude,length,stype,lanes,name
0,308511,50,E,3,17,None,31.627,60.162,38.761062,-120.569835,5.0,ML,2,Sly Park Rd


In [13]:
snap["total_flow"]=snap["total_flow"].astype(float)

In [14]:
#create new column with fwy and direction as string
snap["fwdir"] = snap.apply(lambda x: str(x["fwy"]) + str(x["direc"]), axis=1)
snap["fwdir"].unique()

array(['94W', '125S', '78E', '163S', '8W', '8E', '125N', '94E', '805S',
       '78W', '15N', '5S', '15S', '5N', '805N', '163N', '54E', '54W',
       '56E', '56W', '52E', '52W', '67S', '67N', '905W', '905E', '76W',
       '76E', '11E', '11W'], dtype=object)

In [15]:
import pickle
with open("SD_fwys.pkl", "wb") as fid:
    pickle.dump(list(snap["fwdir"].unique()), fid)

In [41]:
fwdir='5S' #'94W'
sel=snap.query('fwdir == "%s"' % fwdir).sort_values(by="abs_pm")
sel.head()

,longitude,latitude,stype,fwy,direc,abs_pm,total_flow,avg_speed,fwdir
sid,,,,,,,,,
1114228,-117.032738,32.544296,ML,5,S,0.221,406.0,68.5,5S
1118326,-117.045806,32.551447,ML,5,S,1.140,169.0,66.7,5S
1118341,-117.061806,32.558188,ML,5,S,2.186,211.0,69.0,5S
1114721,-117.067228,32.561106,ML,5,S,2.559,304.0,66.4,5S
1118365,-117.076731,32.568952,ML,5,S,3.336,281.0,67.7,5S


In [104]:
#function to move fwys in separate directions apart for visualization
def spread_lanes(sdf, fwdir, latspread=0.0005, lonspread=0.0005):
    direc=fwdir[-1]
    
    # this assumes right-hand traffic
    if direc == 'W':
         sdf["latitude"] += latspread
    elif direc == "E":
        sdf["latitude"] -= latspread
    elif direc == "S":
        sdf["longitude"] -= lonspread
    elif direc == "N":
        sdf["longitude"] += lonspread
    
    return sdf
    
selhr=pd.read_sql("select * from usdot u where u.fwdir = '94W' order by u.abs_pm", engine)
display(selhr.head(n=1))
spread_lanes(selhr, '94W').head(n=1)

,usdid,fwdir,latitude,longitude,abs_pm
0,20932,94W,32.713289,-117.142203,0.0


,usdid,fwdir,latitude,longitude,abs_pm
0,20932,94W,32.713789,-117.142203,0.0


In [105]:
hrdfs=[]

from scipy.interpolate import interp1d

for fwdir in snap["fwdir"].unique():
    print("\rInterpolating average speed along %s" % fwdir, end="")
    sel=snap.query('fwdir == "%s"' % fwdir).sort_values(by="abs_pm")
    selhr=pd.read_sql("select * from usdot u where u.fwdir = '%s' order by u.abs_pm" % fwdir, engine)
    selhr=spread_lanes(selhr, fwdir)

    if len(sel) > 1:
        interpmod = interp1d(sel["abs_pm"].values, sel["avg_speed"].values, bounds_error=False)
        selhr["avg_speed"] = interpmod(selhr["abs_pm"].values)
        selhr.dropna(axis="index", how="any", subset=["avg_speed"], inplace=True)
    #selhr["avg_speed"].describe()
        hrdfs.append(selhr)
print(" done.")

selhr=pd.concat(hrdfs)
selhr.reset_index(drop=True, inplace=True)

Interpolating average speed along 11WE done.


In [106]:
def speed2cat(speed):
    if speed > 65:
        return 'fast'
    elif speed > 60:
        return 'moderate'
    elif speed > 50:
        return 'slow'
    else:
        return 'very slow'
        
selhr["speed_cat"]=selhr["avg_speed"].apply(speed2cat)

In [107]:
selhr.head()

,usdid,fwdir,latitude,longitude,abs_pm,avg_speed,speed_cat
0,20951,94W,32.713815,-117.134018,0.475853,65.650778,fast
1,20952,94W,32.713819,-117.133747,0.491610,65.611711,fast
2,20953,94W,32.713818,-117.133693,0.494750,65.603926,fast
3,20954,94W,32.713818,-117.133531,0.504168,65.580576,fast
4,20955,94W,32.713818,-117.133477,0.507307,65.572793,fast


In [108]:
# %load road_group_split.py
def road_group_split(indf, splitcat="speed_cat", 
                     keep_cols=["latitude", "longitude", "abs_pm", "avg_speed","fwdir"]):
    
    odict={}
    for col in keep_cols + [splitcat]:
        odict[col] = []
    #lats=[]
    #lons=[]
    #abs_pm=[]
    #cat=[]
    odict["linegroup"] = []
    
    p=0 #number holding line group
    i0=indf.index[0]
    
    for col in keep_cols + [splitcat]:
        odict[col].append(indf.loc[i0,col])
    
    #lats.append(indf.loc[i0, "latitude"])
    #lons.append(indf.loc[i0, "longitude"])
    #abs_pm.append(indf.loc[i0, "abs_pm"])
    #cat.append(indf.loc[i0, splitcat])
    #group.append(p)
    odict["linegroup"].append(p)
    
    for n in indf.index[1:]:
        if indf.loc[n,"fwdir"] != indf.loc[i0,"fwdir"]:
            p+=1
            
        if indf.loc[n,splitcat] != indf.loc[i0,splitcat] and indf.loc[n,"fwdir"] == indf.loc[i0,"fwdir"]:
            for col in keep_cols:
                odict[col].append(indf.loc[n,col])
            odict[splitcat].append(indf.loc[i0,splitcat])
            #end previous group with new point and speed cat from previous point
            #lats.append(indf.loc[n,"latitude"])
            #lons.append(indf.loc[n,"longitude"])
            #lons.append(indf.loc[n,"abs_pm"])
            #cat.append(indf.loc[i0,splitcat])
            #group.append(p)
            odict["linegroup"].append(p)
            p+=1
        #start new group
        for col in keep_cols + [splitcat]:
            odict[col].append(indf.loc[n,col])
        #lats.append(indf.loc[n,"latitude"])
        #lons.append(indf.loc[n,"longitude"])
        #cat.append(indf.loc[n,splitcat])
        #group.append(p)        
        odict["linegroup"].append(p)
        i0=n
    
    #odf=pd.DataFrame({"latitude": lats, "longitude": lons, splitcat: cat, "linegroup": group})
    odf=pd.DataFrame(odict)
    return odf

selhrg=road_group_split(selhr)
selhrg.tail()

,latitude,longitude,abs_pm,avg_speed,fwdir,speed_cat,linegroup
13102,32.564593,-116.942441,0.262611,51.544628,11W,slow,337
13103,32.564564,-116.940787,0.358946,51.276353,11W,slow,337
13104,32.564560,-116.940534,0.373681,51.235319,11W,slow,337
13105,32.564519,-116.936522,0.607322,54.121967,11W,slow,337
13106,32.564504,-116.934870,0.703525,58.255156,11W,slow,337


In [109]:
cdm={"fast": "green",
     "moderate": "orange",
     "slow": "red",
     "very slow": "brown"}

In [110]:
#del fig
fig = px.line_mapbox(selhrg, lat="latitude", lon="longitude", color="speed_cat",
                        mapbox_style="stamen-terrain",
                        #color_continuous_scale="aggrnyl" ,
                        line_group = "linegroup",
                        hover_data=["abs_pm", "avg_speed"],
                        center = {"lat": 32.77, "lon": -117.16},
                        color_discrete_map= cdm,
                        zoom=12,
                        height=800, width=800)

for dat in fig.data:
    dat["line"]["width"] = 3

fig2 = px.scatter_mapbox(sel, lat="latitude", lon="longitude",
                        hover_data=["stype","fwy","direc","abs_pm","total_flow","avg_speed"],
                         color="stype",
                        color_discrete_map={"ML": "white"},
                        opacity=0.35)
fig.add_trace(fig2.data[0])
fig
#px.scatter(sel, x="longitude", y="latitude", color="total_flow").update_traces(mode='lines+markers')

In [111]:
fig.write_image("traffic_layer_w_usdot.jpg")

In [88]:
px.line_mapbox?

In [174]:
fig2.data[0].on_click(cfun)

In [56]:
import plotly.graph_objects as go

In [99]:
go.Scattermapbox?

In [127]:
fig2=px.scatter_mapbox(sel, lat="latitude", lon="longitude", mapbox_style="carto-darkmatter")


#tr=go.Scattermapbox(lat=selhr["latitude"], lon=selhr["longitude"],
#        mode='markers')
        #marker=go.scattermapbox.Marker(size=14))
#fig2.add_trace(tr)
#fig2.update_layout()
fig2

In [129]:
go.Scattermapbox?